In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg, col,monotonically_increasing_id 

spark = SparkSession.builder.appName("Gold Layer").getOrCreate()

In [0]:
silver_sellers = spark.read.format("delta").load("/mnt/delta/tables/silver/sellers")
silver_buyers = spark.read.format("delta").load("/mnt/delta/tables/silver/buyers")
silver_users = spark.read.format("delta").load("/mnt/delta/tables/silver/users")
silver_countries = spark.read.format("delta").load("/mnt/delta/tables/silver/countries")

In [0]:
# Perform the join operations
comprehensive_user_table = silver_users \
    .join(silver_countries, ["country"], "outer") \
    .join(silver_buyers, ["country"], "outer") \
    .join(silver_sellers, ["country"], "outer")

# Select and alias columns from each dataframe to ensure uniqueness
comprehensive_user_table = comprehensive_user_table.select(
    silver_users["country"].alias("Country"),
    # From silver_users
    silver_users["productsSold"].alias("Users_productsSold"),
    silver_users["productsWished"].alias("Users_productsWished"),
    silver_users["seniorityasyears"].alias("Users_account_age_years"),
    silver_users["account_age_group"].alias("Users_account_age_group"),
    silver_users["hasanyapp"].alias("Users_hasanyapp"),
    silver_users["socialnbfollowers"].alias("Users_socialnbfollowers"),
    
    # From silver_countries
    silver_countries["sellers"].alias("Countries_Sellers"),
    silver_countries["topsellers"].alias("Countries_TopSellers"),
    silver_countries["femalesellers"].alias("Countries_FemaleSellers"),
    silver_countries["malesellers"].alias("Countries_MaleSellers"),
    silver_countries["topfemalesellers"].alias("Countries_TopFemaleSellers"),
    silver_countries["topmalesellers"].alias("Countries_TopMaleSellers"),
    
    # From silver_buyers
    silver_buyers["buyers"].alias("Buyers_Total"),
    silver_buyers["topbuyers"].alias("Buyers_Top"),
    silver_buyers["femalebuyers"].alias("Buyers_Female"),
    silver_buyers["malebuyers"].alias("Buyers_Male"),
    silver_buyers["topfemalebuyers"].alias("Buyers_TopFemale"),
    silver_buyers["topmalebuyers"].alias("Buyers_TopMale"),
    
    # From silver_sellers
    silver_sellers["nbsellers"].alias("Sellers_Total"),
    silver_sellers["sex"].alias("Sellers_Sex"),
    silver_sellers["meanproductssold"].alias("Sellers_MeanProductsSold"),
    silver_sellers["meanproductslisted"].alias("Sellers_MeanProductsListed"),
)


In [0]:
comprehensive_user_table.show(5)

+---------+------------------+--------------------+-----------------------+-----------------------+---------------+-----------------------+-----------------+--------------------+-----------------------+---------------------+--------------------------+------------------------+------------+----------+-------------+-----------+----------------+--------------+-------------+-----------+------------------------+--------------------------+
|  Country|Users_productsSold|Users_productsWished|Users_account_age_years|Users_account_age_group|Users_hasanyapp|Users_socialnbfollowers|Countries_Sellers|Countries_TopSellers|Countries_FemaleSellers|Countries_MaleSellers|Countries_TopFemaleSellers|Countries_TopMaleSellers|Buyers_Total|Buyers_Top|Buyers_Female|Buyers_Male|Buyers_TopFemale|Buyers_TopMale|Sellers_Total|Sellers_Sex|Sellers_MeanProductsSold|Sellers_MeanProductsListed|
+---------+------------------+--------------------+-----------------------+-----------------------+---------------+-----------

In [0]:
comprehensive_user_table.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Users_productsSold: integer (nullable = true)
 |-- Users_productsWished: integer (nullable = true)
 |-- Users_account_age_years: decimal(10,2) (nullable = true)
 |-- Users_account_age_group: string (nullable = true)
 |-- Users_hasanyapp: boolean (nullable = true)
 |-- Users_socialnbfollowers: integer (nullable = true)
 |-- Countries_Sellers: integer (nullable = true)
 |-- Countries_TopSellers: integer (nullable = true)
 |-- Countries_FemaleSellers: integer (nullable = true)
 |-- Countries_MaleSellers: integer (nullable = true)
 |-- Countries_TopFemaleSellers: integer (nullable = true)
 |-- Countries_TopMaleSellers: integer (nullable = true)
 |-- Buyers_Total: integer (nullable = true)
 |-- Buyers_Top: integer (nullable = true)
 |-- Buyers_Female: integer (nullable = true)
 |-- Buyers_Male: integer (nullable = true)
 |-- Buyers_TopFemale: integer (nullable = true)
 |-- Buyers_TopMale: integer (nullable = true)
 |-- Sellers_Total: integer 

In [0]:
comprehensive_user_table.write.format("delta").mode("overwrite").save("/mnt/delta/tables/goldecom_one_big_table")